# Divergencia Kullback-Leiblich

#### Cargamos el texto para su normalización (preprocesamiento).

In [1]:
nombre_archivo = "e960401_mod.htm"
dir_archivo = "./texto/"
archivo = open(dir_archivo + nombre_archivo, encoding='utf-8')
texto = archivo.read()

#### Quitamos las etiquetas html del texto.

In [2]:
def quitar_html(texto):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(texto, 'lxml')
    return soup.get_text()
texto = quitar_html(texto)

#### Ahora podemos dividir el texto en oraciones.

In [3]:
def obtener_oraciones(texto):
    import nltk
    sent_tokenizer = nltk.data.load('nltk:tokenizers/punkt/english.pickle')
    return sent_tokenizer.tokenize(texto)
oraciones = obtener_oraciones(texto)

#### Ya que tenemos las oraciones hay que realizar varios pasos de normalización. Primero tokenizamos cada oracion. 

In [4]:
def tokenizar_oracion(oracion):
    from nltk.tokenize import word_tokenize
    return word_tokenize(oracion)
oraciones_tokenizadas = [tokenizar_oracion(o) for o in oraciones]

#### Ahora tenemos que etiquetar las partes de cada oracion (POS tagging) con un etiquetador que ya entrenamos. Cargamos el etiquetador.

In [5]:
def cargar_archivo_pickle(nombre):
    import pickle
    with open("./archivo/" + nombre + '.pkl', "rb") as fp:
        arc = pickle.load(fp)
    return arc
eti = cargar_archivo_pickle("tagger")

#### Ahora podemos etiquetar las partes de las oraciones, el etiquetador regresa una lista de tuplas donde el primer elemento es la palabra y el segundo es la parte de la oración que ocupa la palabra.

In [6]:
def etiquetar_oracion(oracion, etiquetador):
    return etiquetador.tag(oracion)
oraciones_etiquetadas = [etiquetar_oracion(o, eti) for o in oraciones_tokenizadas]

#### Ahora tenemos el texto sin etiquetas html, segmentado en oraciones tokenizadas y con cada oración etiquetada, sin embargo, son necesarios más pasos. En este paso vamos a pasar a minúsculas tanto la palabra, como la parte de oración que le corresponde.

In [7]:
def oracion_a_minuscula(oracion):
    return [(tpl[0].lower(), tpl[1][0].lower()) for tpl in oracion]
oraciones_etiquetadas = [oracion_a_minuscula(o) for o in oraciones_etiquetadas]


#### Ahora que las oraciones están etiquetadas y en minúsculas, podemos eliminar cualquier signo de puntación, número o caracter especial del texto.

In [8]:
def quitar_car_especiales(oracion):
    import re
    oracion_limpia = []
    for tpl in oracion:
        cadena_limpia = ""
        for c in tpl[0]:
            if re.match(r'[a-záéíóúñü]', c):
                cadena_limpia += c
        if cadena_limpia != '':
            oracion_limpia.append((cadena_limpia, tpl[1]))
    return oracion_limpia
oraciones_etiquetadas = [quitar_car_especiales(o) for o in oraciones_etiquetadas]

#### Por último quitaremos de cada oración las "stopwords" que son palabras tan comunes que no agregan significado a las oraciones ni al texto en general.

In [9]:
def quitar_stopwords(oracion):
    from nltk.corpus import stopwords
    stopwords_espanol = set(stopwords.words('spanish'))
    return [tpl for tpl in oracion if tpl[0] not in stopwords_espanol]
oraciones_etiquetadas = [quitar_stopwords(o) for o in oraciones_etiquetadas]

#### Ahora podemos lematizar cada oración del texto con la ayuda de un diccionario de lemas.

In [10]:
def lematizar_oracion(oracion, lemas):
    oracion_tokenizada = []
    for t in oracion:
        temp = t[0] + " " + t[1]
        if (temp in lemas):
            oracion_tokenizada.append((lemas[temp], t[1]))
        else:
            oracion_tokenizada.append(t)
    return oracion_tokenizada
lemas = cargar_archivo_pickle("lemmas_dict")
oraciones_etiquetadas = [lematizar_oracion(o, lemas) for o in oraciones_etiquetadas]

#### Ahora obtenemos el vocabulario para proceder a encontrar la entropía condicional de alguna palabra dada

In [11]:
import functools
import operator
vocabulario = functools.reduce(operator.concat, oraciones_etiquetadas)
vocabulario = set(vocabulario)
print("longitud del vocabulario:", len(vocabulario))

longitud del vocabulario: 6798


#### Crearemos un diccionario de en cuántos segmentos aparece cada palabra para su uso más adelante

In [12]:
def diccionario_freq (vocabulario, oraciones_etiquetadas):
    res = {}
    for p in vocabulario:
        count = 0
        for o in oraciones_etiquetadas:
            if p in o:
                count += 1
        res[p] = count        
    return res

# Funcion para guardar el diccionario creado
def guardar_archivo_pickle(filename, obj):
    from pickle import dump
    output = open("archivo/"+filename+".pkl", "wb")
    dump(obj, output, -1)
    output.close()
    return None

#dic_freq = diccionario_freq(vocabulario, oraciones_etiquetadas)
#guardar_archivo_pickle("dic_frec", dic_freq)
dic_frec = cargar_archivo_pickle("dic_frec")

#### Ahora que tenemos las oraciones normalizadas podemos empezar a hacer el cálculo de  la divergencia KL de una palabra en relación con todas las demás:

In [18]:
def divergencia_kl(palabra, oraciones, vocabulario, dic_frec):
    import numpy as np
    n = len(oraciones)
    divergencias = []
    # probabilidad que w1 = 1 usando suavizado
    p1_1 = (dic_frec[palabra] + 0.5) / (n + 1)
    # probabilidad que w1 = 0
    p1_0 = 1 - p1_1
    for p in vocabulario:
        # probabilidad de que w2 = 1, w2 = 0
        # y probabilidad conjunta w1 y w2
        p2_1 = (dic_frec[p] + 0.5) / (n + 1)
        p2_0 = 1 - p2_1
        count_ambas = 0
        for o in oraciones:
            if (p in o) and (palabra in o):
                count_ambas += 1
        p_1_1 = (count_ambas + 0.25) / (n + 1)
        p_0_1 = p2_1 - p_1_1
        p_0_0 = p1_0 - p_0_1
        p_1_0 = p2_0 - p_0_0
        # calculo de la divergencia
        #d_0_0 = 0
        #d_1_0 = 0
        #d_0_1 = 0
        #d_1_1 = 0
        d_0_0 = p_0_0 * np.log2(p_0_0 / (p1_0 * p2_0))
        d_1_0 = p_1_0 * np.log2(p_1_0 / (p1_1 * p2_0))
        d_0_1 = p_0_1 * np.log2(p_0_1 / (p1_0 * p2_1))
        d_1_1 = p_1_1 * np.log2(p_1_1 / (p1_1 * p2_1))
        div = d_0_0 + d_0_1 + d_1_0 + d_1_1
        divergencias.append((p, div))
        divergencias.sort(key = lambda x: x[1], reverse=True)
    return divergencias
palabra = ("presidente", "n")
div = divergencia_kl(palabra, oraciones_etiquetadas, vocabulario, dic_frec)
for d in div:
    print(d)

(('presidente', 'n'), 0.14353441529824848)
(('zedillo', 'n'), 0.026198506475008182)
(('consejo', 'n'), 0.01095771111832453)
(('ernesto', 'n'), 0.010404432053808253)
(('ponce', 's'), 0.008210170393047162)
(('república', 'n'), 0.006984680533202326)
(('león', 's'), 0.006716194772389654)
(('veredicto', 'n'), 0.006716194772389654)
(('pablo', 'n'), 0.006716194772389654)
(('carlos', 'n'), 0.006564643944352226)
(('privado', 'a'), 0.006543147503489244)
(('mes', 'n'), 0.006149200565780178)
(('canal', 'n'), 0.0058723501854876185)
(('alvaro', 'n'), 0.0058723501854876185)
(('manuel', 'n'), 0.0058723501854876185)
(('cabal', 's'), 0.0058723501854876185)
(('peniche', 's'), 0.0058723501854876185)
(('tribunal', 'n'), 0.0058723501854876185)
(('ejército', 'n'), 0.0058723501854876185)
(('trabajadores', 's'), 0.0052721840822806225)
(('obregón', 's'), 0.0052721840822806225)
(('administración', 'n'), 0.005154388629669537)
(('actitud', 'n'), 0.0048059979222343585)
(('procurador', 'n'), 0.004425476557317357)
((

(('coincidente', 's'), 0.00032628974649105994)
(('lleno', 'n'), 0.00032628974649105994)
(('emprendedores', 's'), 0.00032628974649105994)
(('subcoordinador', 's'), 0.00032628974649105994)
(('elaborado', 'a'), 0.00032628974649105994)
(('aprovechada', 'n'), 0.00032628974649105994)
(('liberación', 'n'), 0.00032628974649105994)
(('otorgado', 'n'), 0.00032628974649105994)
(('habientes', 's'), 0.00032628974649105994)
(('afrancisco', 's'), 0.00032628974649105994)
(('origina', 'n'), 0.00032628974649105994)
(('enterarse', 's'), 0.00032628974649105994)
(('seríainnsensato', 'n'), 0.00032628974649105994)
(('romana', 'n'), 0.00032628974649105994)
(('adaptarse', 's'), 0.00032628974649105994)
(('macroeconomía', 'n'), 0.00032628974649105994)
(('caudillesca', 'n'), 0.00032628974649105994)
(('incapacidad', 's'), 0.00032628974649105994)
(('ganador', 'n'), 0.00032628974649105994)
(('modernizar', 'v'), 0.00032628974649105994)
(('tibio', 'a'), 0.00032628974649105994)
(('alfa', 'n'), 0.00032628974649105994)
(

(('doug', 's'), 0.00032628974649105994)
(('volátil', 's'), 0.00032628974649105994)
(('convivía', 'n'), 0.00032628974649105994)
(('casilla', 'n'), 0.00032628974649105994)
(('facultades', 's'), 0.00032628974649105994)
(('mia', 'n'), 0.00032628974649105994)
(('orgullo', 'n'), 0.00032628974649105994)
(('concierto', 'n'), 0.00032628974649105994)
(('confluencia', 'n'), 0.00032628974649105994)
(('concesión', 'n'), 0.00032628974649105994)
(('freno', 'n'), 0.00032628974649105994)
(('fortalecer', 's'), 0.00032628974649105994)
(('caéis', 's'), 0.00032628974649105994)
(('labastida', 'n'), 0.00032628974649105994)
(('difunda', 'n'), 0.00032628974649105994)
(('enmarca', 'n'), 0.00032628974649105994)
(('escolta', 'n'), 0.00032628974649105994)
(('chicomuselo', 's'), 0.00032628974649105994)
(('ocurrirán', 's'), 0.00032628974649105994)
(('requerimos', 'n'), 0.00032628974649105994)
(('allora', 'n'), 0.00032628974649105994)
(('golf', 'n'), 0.00032628974649105994)
(('profundidad', 'n'), 0.000326289746491059

(('vuelta', 'n'), 0.00020697031465234265)
(('compuserve', 's'), 0.00020697031465234265)
(('estabilización', 'n'), 0.00020697031465234265)
(('rojo', 'a'), 0.00020697031465234265)
(('encuentro', 'n'), 0.00020697031465234265)
(('arbitraria', 'n'), 0.00020697031465234265)
(('visita', 'n'), 0.00020697031465234265)
(('convertida', 'a'), 0.00020697031465234265)
(('terminados', 'n'), 0.00020697031465234265)
(('verde', 's'), 0.00020697031465234265)
(('quebraron', 's'), 0.00020697031465234265)
(('combatir', 'v'), 0.00020697031465234265)
(('neumonía', 'n'), 0.00020697031465234265)
(('corrección', 's'), 0.00020697031465234265)
(('triunfal', 'a'), 0.00020697031465234265)
(('mediano', 'a'), 0.00020697031465234265)
(('hormiga', 'n'), 0.00020697031465234265)
(('bando', 'n'), 0.00020697031465234265)
(('dls', 's'), 0.00020697031465234265)
(('dominar', 'v'), 0.00020697031465234265)
(('cobrar', 'v'), 0.00020697031465234265)
(('eviten', 's'), 0.00020697031465234265)
(('jerusalén', 's'), 0.00020697031465234